In [1]:
import pandas as pd
import scipy.sparse as sp
import torch
import torch.nn as nn

device = "mps"

In [2]:
demo_M = 4
demo_N = 5

demo_k = 2

demo_Y_sparse_indices_df = pd.DataFrame([
    [0, 0],
    [0, 2],
    [0, 3],
    [1, 1],
    [1, 4],
    [2, 0],
    [2, 1],
    [2, 2],
    [3, 3],
    [3, 4],
], columns=['user_id', 'item_id'])

def build_Y(sparse_indices_df, _M, _N):
    return sp.coo_matrix(([1.0] * len(sparse_indices_df), (sparse_indices_df['user_id'], sparse_indices_df['item_id'])), shape=(_M, _N))

demo_Y = build_Y(demo_Y_sparse_indices_df, demo_M, demo_N)
demo_Y.todense()

matrix([[1., 0., 1., 1., 0.],
        [0., 1., 0., 0., 1.],
        [1., 1., 1., 0., 0.],
        [0., 0., 0., 1., 1.]])

In [3]:
demo_u = []
demo_i = []
demo_y = []
demo_Y_dense = demo_Y.todense()
for u in range(demo_Y.shape[0]):
    for i in range(demo_Y.shape[1]):
        demo_u.append(u)
        demo_i.append(i)
        demo_y.append([demo_Y_dense[u, i]])

demo_u = torch.tensor(demo_u).to(device)
demo_i = torch.tensor(demo_i).to(device)
demo_y = torch.tensor(demo_y).to(device, dtype=torch.float)

In [15]:
demo_y

tensor([[1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.]], device='mps:0')

In [4]:
class MF(nn.Module):
    def __init__(self, M, N, embedding_dim):
        super(MF, self).__init__()
        
        self.M = M
        self.N = N
        
        self.embedding_dim = embedding_dim
        
        self.P = nn.Embedding(num_embeddings=self.M, embedding_dim=self.embedding_dim)
        self.Q = nn.Embedding(num_embeddings=self.N, embedding_dim=self.embedding_dim)
        
        self.__init_weights()
        
    def __init_weights(self):
        nn.init.normal_(self.P.weight, std=.01)
        nn.init.normal_(self.Q.weight, std=.01)
        
    def forward(self, user_id, item_id):
        p_u = self.P(user_id)
        q_u = self.Q(item_id)
        
        multiplied = torch.multiply(p_u, q_u)
        return torch.sum(multiplied, dim=-1).view(-1, 1)

In [5]:
demo_model = MF(demo_M, demo_N, embedding_dim=2).to(device)
demo_model

MF(
  (P): Embedding(4, 2)
  (Q): Embedding(5, 2)
)

In [6]:
demo_y_hat = demo_model.forward(demo_u, demo_i)
demo_y_hat

tensor([[ 7.2953e-04],
        [-3.9053e-04],
        [-3.2063e-04],
        [ 1.7342e-04],
        [-4.1993e-04],
        [ 2.7221e-04],
        [-1.3812e-04],
        [-1.2292e-04],
        [ 9.3078e-05],
        [-1.6131e-04],
        [ 4.1065e-04],
        [-1.5915e-04],
        [-2.0672e-04],
        [ 3.2406e-04],
        [-2.7322e-04],
        [-8.0520e-05],
        [ 1.5752e-05],
        [ 4.7216e-05],
        [-1.2121e-04],
        [ 6.2953e-05]], device='mps:0', grad_fn=<ViewBackward0>)

In [7]:
demo_loss_function = nn.BCEWithLogitsLoss()
demo_loss_function(demo_y_hat, demo_y).item()

0.6931270956993103

In [8]:
learning_rate = 0.01
demo_optimizer = torch.optim.Adam(demo_model.parameters(), lr=learning_rate)

demo_model.train()

epochs = 100
for i in range(epochs):
    demo_model.zero_grad()
    demo_y_hat = demo_model(demo_u, demo_i)

    loss = demo_loss_function(demo_y_hat, demo_y)
    loss.backward()

    print(loss.item())

    demo_optimizer.step()

0.6931270956993103
0.6930176019668579
0.692825973033905
0.6925336122512817
0.692135751247406
0.6916260719299316
0.6909999251365662
0.6902531981468201
0.6893821358680725
0.6883829236030579
0.6872515082359314
0.6859839558601379
0.6845762133598328
0.6830245852470398
0.6813251376152039
0.6794745326042175
0.6774696111679077
0.6753070950508118
0.6729843020439148
0.6704986691474915
0.6678478717803955
0.6650298833847046
0.6620429158210754
0.6588854193687439
0.6555564403533936
0.6520552635192871
0.6483819484710693
0.6445370316505432
0.6405216455459595
0.6363377571105957
0.6319878697395325
0.6274749636650085
0.622802734375
0.6179754137992859
0.6129974722862244
0.6078738570213318
0.6026100516319275
0.5972116589546204
0.5916852951049805
0.5860379338264465
0.580277144908905
0.5744110941886902
0.568448007106781
0.5623967051506042
0.5562661290168762
0.5500654578208923
0.543803870677948
0.5374908447265625
0.5311356782913208
0.5247476696968079
0.5183362364768982
0.5119102597236633
0.5054787993431091
0.

In [9]:
demo_model.eval()

MF(
  (P): Embedding(4, 2)
  (Q): Embedding(5, 2)
)

In [10]:
demo_model.P.weight

Parameter containing:
tensor([[-1.3109,  0.5936],
        [ 1.2839, -0.6021],
        [-0.6292,  1.3169],
        [ 0.6260, -1.3087]], device='mps:0', requires_grad=True)

In [11]:
demo_model.Q.weight

Parameter containing:
tensor([[-1.1752,  1.1475],
        [ 1.1846,  1.1855],
        [-1.1316,  1.1542],
        [-1.1752, -1.1759],
        [ 1.1629, -1.1511]], device='mps:0', requires_grad=True)

In [12]:
demo_model.P.weight @ demo_model.Q.weight.T

tensor([[ 2.2217, -0.8491,  2.1686,  0.8424, -2.2077],
        [-2.1997,  0.8071, -2.1478, -0.8008,  2.1861],
        [ 2.2506,  0.8159,  2.2320, -0.8092, -2.2475],
        [-2.2374, -0.8099, -2.2189,  0.8032,  2.2343]], device='mps:0',
       grad_fn=<MmBackward0>)